In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
import syft as sy  # import the Pysyft library
hook = sy.TorchHook(torch)  # hook PyTorch ie add extra functionalities 

# IMPORTANT: Local worker should not be a client worker
hook.local_worker.is_client_worker = False


server = hook.local_worker

In [25]:
x11 = torch.tensor([-1, 2.]).tag('input_data')
x12 = torch.tensor([1, -2.]).tag('input_data2')
x21 = torch.tensor([-1, 2.]).tag('input_data')
x22 = torch.tensor([1, -2.]).tag('input_data2')

device_1 = sy.VirtualWorker(hook, id="device_1", data=(x11, x12)) 
device_2 = sy.VirtualWorker(hook, id="device_2", data=(x21, x22))
devices = device_1, device_2

For functions (@ sy.func2plan) we can automatically build the plan with no need to explicitly calling build, actually in the moment of creation the plan is already built.

To get this functionality the only thing you need to change when creating a plan is setting an argument to the decorator called args_shape which should be a list containing the shapes of each argument.

In [26]:
#@sy.func2plan(args_shape=[(-1, 1)])
@sy.func2plan()
def plan_double_abs(x):
    x = x + x
    x = torch.abs(x)
    return x

In [27]:
plan_double_abs

<Plan plan_double_abs id:59312063992 owner:me>

In [28]:
pointer_to_data = device_1.search('input_data')[0]
pointer_to_data

tensor([-1.,  2.])
	Tags: input_data 
	Shape: torch.Size([2])

In [29]:
plan_double_abs.is_built

False

In [30]:
# Sending non-built Plan will fail
try:
    plan_double_abs.send(device_1)
except RuntimeError as error:
    print(error)

A plan needs to be built before being sent to a worker.


In [31]:
plan_double_abs.build(torch.tensor([1., -2.]))

PlaceHolder[Id:36787151705]>tensor([2., 4.])

In [32]:
plan_double_abs.is_built

True

In [33]:
# This cell is executed successfully
pointer_plan = plan_double_abs.send(device_1)
pointer_plan

[PointerPlan | me:78752335679 -> device_1:59312063992]

Running a Plan remotely

In [34]:
pointer_to_result = pointer_plan(pointer_to_data)
print(pointer_to_result)

(Wrapper)>[PointerTensor | me:89309642932 -> device_1:12251816869]


In [35]:
pointer_to_result.get()

tensor([2., 4.])

Towards a concrete example

In [36]:
class Net(sy.Plan):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 3)
        self.fc2 = nn.Linear(3, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=0)

In [37]:
net = Net()
net

<Net Net id:37263999346 owner:me>

In [38]:
net.build(torch.tensor([1., 2.]))

PlaceHolder[Id:46051267288]>tensor([-0.4195, -1.0710], grad_fn=<LogSoftmaxBackward>)

In [39]:
pointer_to_net = net.send(device_1)
pointer_to_net

[PointerPlan | me:98941398944 -> device_1:37263999346]

In [40]:
pointer_to_data = device_1.search('input_data')[0]

In [41]:
pointer_to_result = pointer_to_net(pointer_to_data)
pointer_to_result

(Wrapper)>[PointerTensor | me:18273864137 -> device_1:56782238461]

In [42]:
pointer_to_result.get()

tensor([-0.6909, -0.6954], requires_grad=True)

One major feature that we want to have is to use the same plan for several workers, that we would change depending on the remote batch of data we are considering. In particular, we don't want to rebuild the plan each time we change of worker. Let's see how we do this, using the previous example with our small network.

In [43]:
pointer_to_net_1 = net.send(device_1)
pointer_to_data = device_1.search('input_data')[0]
pointer_to_result = pointer_to_net_1(pointer_to_data)
pointer_to_result.get()

tensor([-0.6909, -0.6954], requires_grad=True)

In [44]:
pointer_to_net_2 = net.send(device_2)
pointer_to_data = device_2.search('input_data')[0]
pointer_to_result = pointer_to_net_2(pointer_to_data)
pointer_to_result.get()

tensor([-0.6909, -0.6954], requires_grad=True)

Note: Currently, with Plan classes, you can only use a single method and you have to name it "forward".